In [1]:
from sklearn.metrics import mean_absolute_error as MSE
from tensorflow.keras.models import Sequential
from cmcrameri import cm
from tqdm import tqdm
import seaborn as sns
from common import *
from models import *

2021-12-15 23:12:08.626189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-15 23:12:08.626219: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Loading our data

In [2]:
df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")

price = df['NO2_price'].values.reshape(-1,1)
fload = df['NO2_load_forecasted'].values.reshape(-1,1)
fgen = df['NO2_generation_forecast'].values.reshape(-1,1)


price_days = seperate_column_to_days(price)
fload_days = seperate_column_to_days(fload)
fgen_days = seperate_column_to_days(fgen)

# Structuring our data

In [3]:
lookbehind = 7
input_width = lookbehind*24
horizon = 24
no_hours = input_width + horizon
stride = 24
hour_in_days = int(no_hours / stride)

price_dataset = []
fload_dataset = []
fgen_dataset = []

for i in range(len(price_days) - hour_in_days+1):
    price_dataset.append(np.concatenate((price_days[i:i+hour_in_days])))
    fload_dataset.append(np.concatenate((fload_days[i:i+hour_in_days])))
    fgen_dataset.append(np.concatenate((fgen_days[i:i+hour_in_days])))

price_dataset = np.array(price_dataset)
fload_dataset = np.array(fload_dataset)
fgen_dataset = np.array(fgen_dataset)

scaler = MinMaxScaler()
price_dataset = scaler.fit_transform(price_dataset[:,:,0])
price_dataset = price_dataset[..., np.newaxis].astype(np.float32)

fload_dataset = scaler.fit_transform(fload_dataset[:,:,0])
fload_dataset = fload_dataset[..., np.newaxis].astype(np.float32)

fgen_dataset = scaler.fit_transform(fgen_dataset[:,:,0])
fgen_dataset = fgen_dataset[..., np.newaxis].astype(np.float32)

dataset_forecast = np.concatenate((price_dataset, fload_dataset, fgen_dataset), axis=2)

n,m,k = dataset_forecast.shape

print(dataset_forecast.shape)


(2145, 192, 3)


# Train test split

In [4]:
train = int(0.7*n)
valid = int(0.9*n)
x_train, X_train = price_dataset[:train, :input_width], dataset_forecast[:train, :input_width]
x_valid, X_valid = price_dataset[train:valid, :input_width], dataset_forecast[train:valid, :input_width]
x_test, X_test = price_dataset[valid:, :input_width], dataset_forecast[valid:, :input_width]

Y = np.empty((n, input_width, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = dataset_forecast[:,step_ahead:step_ahead + input_width, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]

# Setting up the models

## Hyperparameters

In [5]:
# Common
epochs = 50
loss = "mse" 
batch_size = [200, 400, 600, 800]
eta_list = np.logspace(-1, -4, 4)

# Univariate
univ_neurons = 128

# Multivariate
multi_neurons_first = 128
multi_neurons_second = 64
filters = 64
kernel_size = 3
strides = 3


learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                            patience=3, 
                                                            verbose=1, 
                                                            factor=0.5, 
                                                            min_lr=0.000001)
callbacks = [learning_rate_reduction]

# Setting up the univariate SimpleRNN

In [6]:
class RNN(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()

        # RNN architecture
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True, input_shape=[None, 1]))
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)


# return_sequences=True --> (256, 168, 24)
# return_sequences=False --> (256, 24)

## Grid search

In [8]:
columns = ["batch_size", "learning_rate", "epoch", "univ_neurons", "mse_all", "mse_all2"]
columns += [f"mse{i}" for i in range(1,25)]
results = pd.DataFrame(columns=columns)
Y_test_last = Y_test[:,-1]

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
training_losses = pd.DataFrame(columns=epochs_colmns)
validation_losses = pd.DataFrame(columns=epochs_colmns)

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        RNN_model = RNN(eta, univ_neurons, horizon=horizon)

        # Training
        history = RNN_model.fit(x_train, Y_train, 
                      epochs=epochs, batch_size = batch, 
                      shuffle=False, callbacks= callbacks,  
                      validation_data=(x_valid, Y_valid), verbose=0)

        
        # Model evaluation result
        training_losses.loc[i] = history.history["loss"]
        validation_losses.loc[i] = history.history["val_loss"]

        # Prediction
        Y_pred_rnn = RNN_model.predict(x_test)
        Y_pred_rnn_last = Y_pred_rnn[:,-1]     
        
        # Results
        mse_val = MSE(Y_test_last, Y_pred_rnn_last)
        se_all = (Y_test_last - Y_pred_rnn_last) ** 2 
        mse_all = np.mean(se_all, axis=0) # The mean of each column 
        mse_val2 = np.mean(mse_all)

        # Storing results
        res1 = np.array([batch, eta, epochs, univ_neurons, mse_val, mse_val2])
        res2 = np.array(mse_all)
        res = np.concatenate([res1,res2], axis=0)
        results.loc[i] = res
        i += 1

        results.to_csv(f"{REPORT_DATA}univariate_results_best_models.csv")
        training_losses.to_csv(f"{REPORT_DATA}univariate_evaluation_training_losses.csv")
        validation_losses.to_csv(f"{REPORT_DATA}univariate_evaluation_validation_losses.csv")

(215, 168, 24)
(215, 168, 24)
0.13920488719620555


# Setting up the multivariate ConvGRU

In [9]:
class ConvGRU(Sequential):
    def __init__(self, eta, neurons_first, neurons_second, horizon=24):
        super(self.__class__, self).__init__()

        # ConvGRU architecture
        self.add(keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding="valid", input_shape=[None, k]))
        self.add(keras.layers.GRU(neurons_first, return_sequences=True))
        self.add(keras.layers.GRU(neurons_second, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

X_train.shape=(1501, 168, 3)
Y_train.shape=(1501, 168, 24)
X_valid.shape=(429, 168, 3)
Y_valid.shape=(429, 168, 24)
X_train.shape=(1501, 168, 3)
Y_train[:,2::3].shape=(1501, 56, 24)
X_valid.shape=(429, 168, 3)
Y_valid[:,2::3].shape=(429, 56, 24)


## Grid search

In [10]:
columns = ["batch_size", "learning_rate", "epoch", "univ_neurons", "mse_all", "mse_all2"]
columns += [f"mse{i}" for i in range(1,25)]
results = pd.DataFrame(columns=columns)
Y_test_last = Y_test[:,-1]

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
training_losses = pd.DataFrame(columns=epochs_colmns)
validation_losses = pd.DataFrame(columns=epochs_colmns)

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        ConvGRU_model = ConvGRU(eta, multi_neurons_first, multi_neurons_second, horizon=horizon)

        # Training
        history = ConvGRU_model.fit(X_train, Y_train[:,2::strides], 
                          epochs=epochs, batch_size = batch, 
                          shuffle=False, callbacks= callbacks,
                          validation_data=(X_valid, Y_valid[:,2::strides]), verbose=0)
        
        
        # Model evaluation result
        training_losses.loc[i] = history.history["loss"]
        validation_losses.loc[i] = history.history["val_loss"]

        # Prediction
        Y_pred_rnn = ConvGRU_model.predict(X_test)
        Y_pred_rnn_last = Y_pred_rnn[:,-1]     
        
        # Results
        mse_val = MSE(Y_test_last, Y_pred_rnn_last)
        se_all = (Y_test_last - Y_pred_rnn_last) ** 2 
        mse_all = np.mean(se_all, axis=0) # The mean of each column 
        mse_val2 = np.mean(mse_all)

        # Storing results
        res1 = np.array([batch, eta, epochs, univ_neurons, mse_val, mse_val2])
        res2 = np.array(mse_all)
        res = np.concatenate([res1,res2], axis=0)
        results.loc[i] = res
        i += 1

        results.to_csv(f"{REPORT_DATA}multivariate_results_best_models.csv")
        training_losses.to_csv(f"{REPORT_DATA}multivariate_evaluation_training_losses.csv")
        validation_losses.to_csv(f"{REPORT_DATA}multivariate_evaluation_validation_losses.csv")

Epoch 1/2
16/16 [==============================] - 5s 136ms/step - loss: 0.0173 - val_loss: 0.0118
Epoch 2/2
16/16 [==============================] - 2s 99ms/step - loss: 0.0057 - val_loss: 0.0062
0.16341531596554118
